# Install Phantom-Omni

The ROS instructions are: http://wiki.ros.org/phantom_omni

The ROS repository is: https://github.com/danepowell/phantom_omni

Development for Phantom-omni: 
- https://fsuarez6.github.io/projects/geomagic-touch-in-ros/
- https://github.com/bharatm11/Geomagic_Touch_ROS_Drivers
    
Follow the instructions to install the device.